PWD's Wingohocking SWMM Model - runs to test effects of GSI installations

In [1]:
# Run this block to load imports,
# to test mongodb connectivity, and view existing collections
#%reset
import yaml
from swmm_objects import *
from swmm_read_cso_time_series import *

from subprocess import call
from datetime import datetime
import sys
from mongo_credentials import *

# IMPORTANT: Set mongodb database name and credentials function below
###########################################
database = "ewri2017"
db = mongo_credentials_aws2_AEM(database)
###########################################
collectionNames = sorted(db.collection_names())
print collectionNames 

[u'2008annual_1', u'cso_experiments_1', u'july', u'july2008_1']


In [2]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking
# model BEFORE adding GSI to obtain the zero LID runoff and CSO values
from swmm_run_cso import *
#  IMPORTANT:  Set these for each run:
#################################################
collection = 'cso_experiments_1'
swmmInpFile = "wingohocking_PWD.inp"  # initial SWMM .inp file
lidZeroFile = "PWD_wingo_Wakefield_list_zero.yaml"
################################################
zeroLidDocId = zero_lid_runswmm(db,collection,swmmInpFile,lidZeroFile)
thisRunCollection = db[collection]
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

Elapsed Time = 0.0 min, 18.39 sec
Stored record 583ba063c1015718b3a1c466 for zero LID usage:  peak = 1906.45, volume = 183.249


In [3]:
from swmm_run_cso import calculate_cso
from mongodb_tools import mongo_doc_id_from_string
#zeroLidDocIdString = "583b648dc10157149703b482"
#zeroLidDocId = mongo_doc_id_from_string(zeroLidDocIdString)
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
outflowSeries = cursor["outflow_series"]
resultDict = calculate_cso(3122.0,outflowSeries,0.05,15.0)
resultDict

{'cso_volume': 2286913269.1718516, 'tot_volume': 6865825056.036434}

### Loop ALL subcatchment LID numbers over a range:

In [ ]:
# Plot Runoff Timeseries


In [3]:
# Now, apply a range of Wakefield Park rain gardens in the selected seven subcatchments simultaneously
from copy import deepcopy
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidList = cursor["runParamList"]
swmmInpStr = cursor["swmmInputFileStr"]
for numberForAllLid in range(9,10):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    print "Number of Wakefield LID in each selected subcatchment $s" % numberForAllLid
    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"

KeyError: <class 'swmm_objects.lid_usage'>

### To display all of the runs from the database:

In [3]:
runs = db[collection]
cursor = runs.find()
#\for run in cursor:
#    print run
#    print ''

### To Reconstitute the SWMM input file for any of the runs above

In [4]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
#print swmmInputFileStr
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()